In [49]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
import re
import MeCab

In [50]:
def separate_languages(text):
    japanese_text = ''
    english_text = ''
    english_word = ''
    include_words = ['dp', 'DP', 'rep', 'if', 'max', 'min', 'vector', 'push_back', 'map', '%', 'size', 'tuple', 'vector', '.second', '.first', 'lower_bound', 'Union', 'Find', 'union', 'find', 'swap', 'sort', 'reverse', 'while', 'abs', 'DFS', 'dfs', 'length', 'priority_queue', 'pop', 'empty', 'top', 'push', 'imos', 'gcd', 'mod', 'XOR', 'BFS', 'bfs', 'C', 'UF', 'que', 'GCD', 'bit', 'permutation']
    exclude_words = ["cin", "cout", "int", "double", "main", "void", "printf", "include", "std", "using", "namespace", "std", "ans", "string", "else", "stdc", "Answer", "answer", "endl", "return", "No", "no", "NO", "_main", "bits", "Editorial", "by", "define", "hamayanhamayan","Tweet","AtCoder","Beginner","Contest","Oddness","Security","JapanTechNews","Kaggle","Windows","Tips","portal","site","Twitter","Writeups","corCTF","TFC","CTF","OSEP","ImaginaryCTF", "scanf", "ABC", "Yay", "%d", "TumoiYorozu"]

    for char in text:
        if '\u3000' <= char <= '\u9FFF' or '\u3040' <= char <= '\u309F' or '\u30A0' <= char <= '\u30FF':
            japanese_text += char
            if english_word and (english_word in include_words or (len(english_word) > 1 and english_word == '%')) and english_word not in exclude_words:
                english_text += english_word + ','
            english_word = ''
        elif re.match(r'[A-Za-z%\_&\|]', char):
            english_word += char
        else:
            if english_word and (english_word in include_words or (len(english_word) > 1 and english_word == '%')) and english_word not in exclude_words:
                english_text += english_word + ','
            english_word = ''
    
    if english_word and (english_word in include_words or (len(english_word) > 1 and english_word == '%')) and english_word not in exclude_words:
        english_text += english_word

    # 末尾のカンマを削除
    if english_text and english_text[-1] == ',':
        english_text = english_text[:-1]

    return japanese_text, english_text


# def separate_languages(text):
#     japanese_text = ''
#     english_text = ''
#     english_word = ''
#     exclude_words = ["cin", "cout", "int", "double", "main", "void", "printf", "include", "std", "using", "namespace", "std", "ans", "string", "else", "stdc", "Answer", "answer", "endl", "return", "No", "no", "NO", "_main", "bits", "Editorial", "by", "define", "hamayanhamayan","Tweet","AtCoder","Beginner","Contest","Oddness","Security","JapanTechNews","Kaggle","Windows","Tips","portal","site","Twitter","Writeups","corCTF","TFC","CTF","OSEP","ImaginaryCTF", "scanf", "ABC", "Yay", "%d", "TumoiYorozu"]

#     for char in text:
#         if '\u3000' <= char <= '\u9FFF' or '\u3040' <= char <= '\u309F' or '\u30A0' <= char <= '\u30FF':
#             japanese_text += char
#             if english_word and (len(english_word) > 1 or english_word == '%') and english_word not in exclude_words:
#                 english_text += english_word + ','
#             english_word = ''
#         elif re.match(r'[A-Za-z%\_&\|.]', char):
#             english_word += char
#         else:
#             if english_word and (len(english_word) > 1 or english_word == '%') and english_word not in exclude_words:
#                 english_text += english_word + ','
#             english_word = ''
    
#     if english_word and (len(english_word) > 1 or english_word == '%') and english_word not in exclude_words: # テキストの最後が英語の場合、追加
#         english_text += english_word

#     # 末尾のカンマを削除
#     if english_text and english_text[-1] == ',':
#         english_text = english_text[:-1]

#     return japanese_text, english_text

In [51]:
def separate_languages_test(text):
    japanese_text = ''
    english_words = set()
    english_word = ''
    exclude_words = ["cin", "cout", "int", "double", "main", "void", "printf", "include", "std", "using", "namespace", "std", "ans", "string", "else", "stdc", "Answer", "answer", "endl", "return", "No", "no", "NO", "_main", "bits", "Editorial", "by", "define", "hamayanhamayan","Tweet","AtCoder","Beginner","Contest","Oddness","Security","JapanTechNews","Kaggle","Windows","Tips","portal","site","Twitter","Writeups","corCTF","TFC","CTF","OSEP","ImaginaryCTF", "scanf", "ABC", "Yay", "%d", "TumoiYorozu"]

    for char in text:
        if '\u3000' <= char <= '\u9FFF' or '\u3040' <= char <= '\u309F' or '\u30A0' <= char <= '\u30FF':
            japanese_text += char
            if english_word and (len(english_word) > 1 or english_word == '%') and english_word not in exclude_words:
                english_words.add(english_word)
            english_word = ''
        elif re.match(r'[A-Za-z%\_&\|.]', char):
            english_word += char
        else:
            if english_word and (len(english_word) > 1 or english_word == '%') and english_word not in exclude_words:
                english_words.add(english_word)
            english_word = ''
    
    if english_word and (len(english_word) > 1 or english_word == '%') and english_word not in exclude_words: # テキストの最後が英語の場合、追加
        english_words.add(english_word)

    english_text = ','.join(english_words)

    return japanese_text, english_text


In [52]:
def tokenize_japanese(text):
    included_keywords = ['最', '和', '遷移', 'ハッシュ', '全', '探索', '区間', 'クエリ', '累積', '貪欲', '単調', '公約', 'ユークリッド', '互除', '二', '分', '倍数', '再帰', '包', '原理', '進', '二', '項', '尺', '動的', '計画', '隣', '座標', '圧縮', 'グラフ', '頂点', '辺', '回転', '幅', '深', '優先', '連結', '成分', '時計', '木', '進', '論理', '素', '約数', 'ラン', 'レングス', '経路', 'セグ', 'ワーシャルフロイド', 'キュー', 'エラトステネス' ]
    mecab = MeCab.Tagger("-Owakati")
    wakati_text = mecab.parse(text).strip()
    tokens = wakati_text.split()
    tokens = [token for token in tokens if any(keyword in token for keyword in included_keywords) and not re.match(r'^[\u3040-\u309F]+$', token)]
    return ','.join(tokens)

# def tokenize_japanese(text):
#     excluded_list = ['、', '。','問題', '解説', '考察', 'コード', '答え', '実装','自分', '入門','中高生','レベル','想定','方法','書き','方']
#     mecab = MeCab.Tagger("-Owakati")
#     wakati_text = mecab.parse(text).strip()
#     tokens = wakati_text.split()
#     tokens = [token for token in tokens if token not in excluded_list and not re.match(r'^[\u3040-\u309F]+$', token)]
#     return ','.join(tokens)

In [53]:
def tokenize_ja_test(text):
    excluded_list = ['、', '。','問題', '解説', '考察', 'コード', '答え', '実装','自分', '入門','中高生','レベル','想定','方法','書き','方']
    mecab = MeCab.Tagger("-Owakati")
    wakati_text = mecab.parse(text).strip()
    tokens = wakati_text.split()
    tokens = [token for token in tokens if token not in excluded_list and not re.match(r'^[\u3040-\u309F]+$', token)]
    unique_tokens = []
    seen = set()
    for token in tokens:
        if token not in seen:
            seen.add(token)
            unique_tokens.append(token)
    return ','.join(unique_tokens)

In [54]:

df = pd.read_csv('outputD_1.csv')

urls = df['url'].tolist()
ja_texts = []
en_texts = []
abc = 2

for url in urls:
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser') # パーサーを指定
    plain_text = soup.get_text()
    if "D - " in plain_text:
        text_after_keyword = plain_text.split("D - ", 1)[1]

        # "hamayanhamayan"があれば、その位置でテキストを終了
        if "posted:" in text_after_keyword:
            text_after_keyword = text_after_keyword.split("posted:", 1)[0]

        clean_text = "\n".join([line for line in text_after_keyword.split('\n') if line.strip() != ''])
    elif "解法" in plain_text:
        text_after_keyword = plain_text.split("解法", 1)[1]

        # "hamayanhamayan"があれば、その位置でテキストを終了
        if "hamayanhamayan" in text_after_keyword:
            text_after_keyword = text_after_keyword.split("hamayanhamayan", 1)[0]

        clean_text = "\n".join([line for line in text_after_keyword.split('\n') if line.strip() != ''])
    elif "解説" in plain_text:
        text_after_keyword = plain_text.split("解説", 1)[1]

        # "hamayanhamayan"があれば、その位置でテキストを終了
        if "hamayanhamayan" in text_after_keyword:
            text_after_keyword = text_after_keyword.split("hamayanhamayan", 1)[0]

        clean_text = "\n".join([line for line in text_after_keyword.split('\n') if line.strip() != ''])
    else:
        print("解説のキーワードが見つかりませんでした。")
        clean_text = ''
    clean_text = re.sub(r'http\S+', '', clean_text)
    japanese, english = separate_languages(clean_text)
    #japanese, english = separate_languages_test(clean_text)
    japanese = tokenize_japanese(japanese)
    #japanese = tokenize_ja_test(japanese)
    ja_texts.append(japanese)
    en_texts.append(english)
    print(abc)
    abc += 1
    sleep(6)

# 新しいカラムとしてclean_textsをDataFrameに追加
df['ja_text'] = ja_texts
df['en_text'] = en_texts
    

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179


In [55]:
# 結果を新しいCSVファイルとして保存、区切り文字としてセミコロンを使用
df.to_csv('outputD_2.csv', index=False, sep=';')